# Preprocessing
Flight data and Weather data at selected airports for the years 2016 and 2017 were merged separately, followed by merging both the dataframes into a single dataframe where each flight record contains the corresponding weather data and saved into a csv file. 

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import pandas as pd

airport_ids=[11298, 14747, 12478, 13930, 14107, 12892, 13204, 11292, 13303, 12266, 12889, 14771, 10397, 11057, 11618]
airport_names=['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
global flight_dataframe
flight_dataframe=pd.DataFrame(columns=['FlightDate','Quarter','Month','Year','DayofMonth','DepTime',
                               'DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','Origin',
                               'DestAirportID','Dest','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes','Cancelled','Diverted'])
for i in ["2016","2017"]:
    basic_url="gdrive/My Drive/Flight Data/" + i+ "/On_Time_On_Time_Performance_" + i + "_"
    for j in range (1,13):
        filenum=str(j)+"/On_Time_On_Time_Performance_" + i+"_"+str(j)+".csv"
        url=basic_url+filenum
        print (url)
        data=pd.read_csv(url,usecols=['FlightDate','Quarter','Month','Year','DayofMonth','DepTime',
                               'DepDel15','CRSDepTime','DepDelayMinutes','OriginAirportID','Origin',
                               'DestAirportID','Dest','ArrTime','CRSArrTime','ArrDel15','ArrDelayMinutes','Cancelled','Diverted'])
        data=data[data['Cancelled']==0]
        data=data[data['Diverted']==0]
        data=data[data.Dest.isin(airport_names)&data.Origin.isin(airport_names)]
        flight_dataframe=pd.concat([flight_dataframe,data])
        print (flight_dataframe.shape[0])
flight_dataframe=flight_dataframe.dropna()
print (flight_dataframe.head(10))


In [0]:
import pandas as pd
import json
airport_names=['ATL','CLT','DEN','DFW','EWR','IAH','JFK','LAS','LAX','MCO','MIA','ORD','PHX','SEA','SFO']
global weather_dataframe
weather_dataframe=pd.DataFrame(columns=['windspeedKmph','winddirDegree','weatherCode','cloudcover','precipMM',
                                        'pressure','WindChillF','DewPointF','tempF','WindGustKmph','visibility','humidity'
                                        ,'time','date','airport'])
for name in airport_names:
  basic_URL="gdrive/My Drive/weather/"+name+"/"
  for year in ["2016", "2017"]:
    for month in range (1,13):
      URL=basic_URL+year+"-"+str(month)+".json"
      print (URL)
      jsonfile=pd.read_json(URL)
      weather_data=jsonfile['data']['weather']
      for i in range(len(weather_data)):
        data=pd.DataFrame(weather_data[i]["hourly"], columns=['windspeedKmph','winddirDegree','weatherCode','cloudcover','precipMM',
                                        'pressure','WindChillF','DewPointF','tempF','WindGustKmph','visibility','humidity','time'])
        data["date"]=weather_data[i]["date"]
        data["airport"]=name
        weather_dataframe=pd.concat([weather_dataframe,data])
    print (weather_dataframe.shape[0])
print (weather_dataframe.head(6))

In [0]:
import pandas as pd
import numpy as np

def approx_time(time):
  time_str=str(int(time))
  minutes='00'
  if len(time_str)>2:
    hour=0
    
    if int(time_str[len(time_str)-2:])>30:
      if int(time_str[:-2])==23:
        hour=23
      else:
        hour=int(time_str[:-2])+1
    else:
        hour=int(time_str[:-2])
  else:
    if int(time_str)>30:
      hour=1
    else:
      hour=0
  return int(str(hour)+minutes)


flight_dataframe["ArrTimeApprox"]=np.array([approx_time(x) for x in flight_dataframe["ArrTime"].tolist()])

In [0]:
weather_dataframe['time']=weather_dataframe['time'].astype(str).astype(int)
#merging_dataframes
global dataframe
dataframe=pd.merge(flight_dataframe, weather_dataframe, left_on=["FlightDate","ArrTimeApprox","Dest"], 
                   right_on=["date","time","airport"],how='left')
pd.set_option('display.max_columns', None)
dataframe= dataframe.drop(['airport','ArrTimeApprox','date','Cancelled','Diverted'],axis=1)
dataframe=dataframe.dropna()
dataframe.to_csv("/content/gdrive/My Drive/file.csv")